# 0.Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install transformers
%pip install -q transformers
! pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.7 MB/s eta 0:00:00


In [3]:
import json
import re
import torch
import time
import logging
import torch
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from transformers.data.processors.squad import SquadExample, squad_convert_examples_to_features
from transformers import AlbertForQuestionAnswering, TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup, AlbertTokenizer
from transformers import TFAutoModel, AutoTokenizer, AutoModel
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from tqdm import trange, tqdm

# 1.AIHUB 데이터 처리

In [4]:
root_dir = '/content/drive/MyDrive/절미네 가족들/1.jiwon/'

In [5]:
# JSON 파일 로드
with open(root_dir + "1.data/0.row_data/AI_hub_fin_data.json", "r", encoding="utf-8-sig") as f:
    data = json.load(f)

In [6]:
# "class"에서 "금융"을 포함하는 데이터만 추출
filtered_data = [item for item in data["data"] if "금융" in item["doc_class"]["class"]]

In [7]:
# filtered_data 중에서 나머지 항목들도 가져오는 코드
def get_fin_data(filtered_data) :

  rows = []

  for item in filtered_data:
      for para in item["paragraphs"]:
          context = para["context"]
          context_id = para["context_id"]
          for qa in para["qas"]:
              question = qa["question"]
              answer_text = qa["answer"]["text"]
              answer_start = qa["answer"]["answer_start"]
              answer_end = qa["answer"]["answer_end"]
              rows.append({
                  'context': context,
                  'context_id': context_id,
                  'question': question,
                  'answer_text': answer_text,
                  'answer_start': answer_start,
                  'answer_end': answer_end
              })

  return rows

In [ ]:
# pd.set_option('display.max_colwidth',None)
# # pd.reset_option('display.max_colwidth')

In [8]:
# 필터된 데이터만 가져갔어
fin_data = get_fin_data(filtered_data)
fin_data_df = pd.DataFrame(fin_data)

In [9]:
fin_data_df2 = fin_data_df[['context_id','context','question','answer_text','answer_start','answer_end']]

In [10]:
fin_data_df2

,context_id,context,question,answer_text,answer_start,answer_end
0,2415370,`22.3월말 보험회사 대출채권 현황\n1. 개요\n□22.3월말 보험회사 대출채권...,경기 변동성이 커지는 것에 대응하기 위해 어떻게 해,연체율 등 대출 건전성 지표를 지속 모니터링,432,455
1,2415370,`22.3월말 보험회사 대출채권 현황\n1. 개요\n□22.3월말 보험회사 대출채권...,어떤 방법으로 경기 변동성이 커지는 것을 대처해,연체율 등 대출 건전성 지표를 지속 모니터링,432,455
2,2415371,"2. 보험영업(수입보험료)\n□ 생보사：120조 5,457억원으로 전년 대비 9,5...",보험회사가 커지는 불확실성에 대해 어떻게 준비해야 돼,장기 수익성 및 자산운용 리스크 관리를 강화하고 재무건전성을 확보,466,501
3,2415371,"2. 보험영업(수입보험료)\n□ 생보사：120조 5,457억원으로 전년 대비 9,5...",불확실성이 커지는 보험회사는 어떤 방법으로 대비를 해야 돼,장기 수익성 및 자산운용 리스크 관리를 강화하고 재무건전성을 확보,466,501
4,2415372,□금융감독원은 ｢2022년 금융감독 온라인 업무설명회｣를 유튜브 및 페이스북을 통해...,금감원은 어떤 방법으로 온라인 업무설명회를 열기로 했어,유튜브 및 페이스북을 통해,32,45
...,...,...,...,...,...,...
18755,2699778,5. 지역의 새로운 금융지원체제 확충방안\n창업·투자 활성화를 위한 지역의 금융지원...,보증 중심의 금융관행에서 탈피하여 어떤 조치를 취해서 기술을 기반으로 한 사업화자금...,기술금융제도를 재편,231,240
18756,2699779,크라우드펀딩을 통한 자금조달\n크라우드펀딩은 아이디어만으로 자금조달이 가능하며 아이...,투자자가 금융사기 때문에 손해를 입을 가능성을 감소시키려면 어떻게 해야 돼,크라우드펀딩플랫폼 관리자들의 적격투자대상 창조기업 선정 노력 등의 모니터링이 강화,467,511
18757,2699779,크라우드펀딩을 통한 자금조달\n크라우드펀딩은 아이디어만으로 자금조달이 가능하며 아이...,어떤 방법으로 투자자가 금융사기로 인한 손실을 볼 가능성을 줄일 수 있어,크라우드펀딩플랫폼 관리자들의 적격투자대상 창조기업 선정 노력 등의 모니터링이 강화,467,511
18758,2699780,"셋째, 코넥스(KONEX)시장 상장을 창조기업의 자금조달 수단으로 충분히 활용할 필...",여러 가지 무형자산들을 어떻게 원활하게 상업화시킬 수 있어,적절한 가치평가를 통해,430,441


In [ ]:
#  fin_data_df2.to_csv('/content/drive/MyDrive/절미네 가족들/1.jiwon/1.data/1.first_cleansing/AI_hub_QA_before.csv',index=False)

# 2.전처리

In [ ]:
fin_data_df3 = fin_data_df2.copy()

In [ ]:
# 데이터 타입을 문자열로 변환
fin_data_df3['context'] = fin_data_df3['context'].astype(str)

# 정규표현식 패턴들
patterns = [
    (r'\n', ' '),                    # \n 삭제 후 공백으로 대체
    (r'[^a-zA-Z0-9가-힣\s.,+\-%()"\'·:]', ''),    # 특수기호 삭제하지 않을 기호 추가
]

# 정규표현식 패턴을 순회하며 처리
for pattern, replacement in patterns:
    fin_data_df3['context'] = fin_data_df3['context'].apply(lambda x: re.sub(pattern, replacement, x))

In [ ]:
fin_data_df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18760 entries, 0 to 18759
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   context_id    18760 non-null  object
 1   context       18760 non-null  object
 2   question      18760 non-null  object
 3   answer_text   18760 non-null  object
 4   answer_start  18760 non-null  int64 
 5   answer_end    18760 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 879.5+ KB


In [ ]:
# fin_data_df3.to_csv(root_dir+'1.data/1.first_cleansing/AI_hub_QA.csv', index = False)

# 3.QA셋 제작

In [11]:
df = pd.read_csv('/content/drive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/QA_set_final_ver2.csv')

In [12]:
df['question_id'] = range(0, len(df))
df['title'] = range(0, len(df))

In [17]:
df = df[['question','context','answer','answer_start','answer_end']]

In [18]:
df

,question,context,answer,answer_start,answer_end
0,생산물시장이 뭐야?,경제활동은 우리 생활에 필요한 재화와 서비스를 생산 · 분배 · 소비하는 활동을 말...,생활에 필요한 재화와 서비스가 거래되는 시장,237.0,261.0
1,생산물시장에서 공급자는 누구야?,경제활동은 우리 생활에 필요한 재화와 서비스를 생산 · 분배 · 소비하는 활동을 말...,기업은 생산물시장에서는 공급자,118.0,134.0
2,정부가 시장 경제를 위해 뭘 해?,"정부는 국방, 치안, 사법제도를 유지하고 국·공립학교의 운영과 같은 수많은 공공서비...","정부는 국방, 치안, 사법제도를 유지하고 국·공립학교의 운영과 같은 수많은 공공서비...",0.0,51.0
3,직접투자가 뭐야?,"우리 기업이 생산한 생산물을 외국으로 수출하기도 하고 석유, 식량, 원자재 같은 것...",회사를 설립하거나 인수하여 직접 운영,221.0,241.0
4,직접투자의 장점이 뭐가 있을까?,"우리 기업이 생산한 생산물을 외국으로 수출하기도 하고 석유, 식량, 원자재 같은 것...",직접 투자는 자본이 들어오는 것뿐만 아니라 기술도 같이 들어오는 장점이 있다,255.0,297.0
...,...,...,...,...,...
1080,주민번호 도용을 확인할 수 있는 무료 방법은 무엇인가?,"NICE평가정보의 마이크레딧, SCI(서울신용평가정보)의 사이렌24, KCB(코리아...",주민번호 클린센터(clean.kisa.or.kr)를 이용하여 주민번호 이용내역을 조...,130.0,184.0
1081,금융소비자 보호에서 가장 큰 이슈로 부각되고 있는 것은 무엇인가요?,최근 금융소비자 보호에서 가장 큰 이슈로 부각되고 있는 것 중 하나가 금융사기이다....,금융사기,39.0,43.0
1082,어떤 현실을 이용한 대학생 대상 사기가 증가하고 있나요?,최근 금융소비자 보호에서 가장 큰 이슈로 부각되고 있는 것 중 하나가 금융사기이다....,"취업난, 등록금 부담 등 어려운 현실을 이용한 대학생 대상 사기",93.0,128.0
1083,"피해예방을 위해 어떤 경우에도 금전의 이체를 요구하거나, 금융거래정보를 수집하지 말...",이와 같이 다양한 금융사기에 피해를 입지 않으려면 아래와 같은 피해예방 요령을 명심...,공공기관 및 금융회사,60.0,71.0


In [ ]:
# df.to_csv('/content/drive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/QA_set_final_ver3.csv', index=False)

# 4.QA셋 코드 수정

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/QA_set_final_ver3.csv')

In [ ]:
def create_examples(input_data1,input_data2, set_type):
    is_training = set_type == "train"
    examples = []

    print("input_data1 처리 시작")
    filtered_data = [item for item in input_data1 if "금융" in item["doc_class"]["class"]]

    for item in filtered_data:
        title = item['doc_title']

        for para in item["paragraphs"]:
            context_text = para["context"]

            for qa in para["qas"]:
                qas_id = qa["question_id"]
                question_text = qa["question"]
                answer_text = qa["answer"]["text"]
                start_position_character = qa["answer"]["answer_start"]
                end_position_character = qa["answer"]["answer_end"]
                answers = []

                is_impossible = qa.get("is_impossible", False)

                # 정답 텍스트와 시작 위치를 가져와서 딕셔너리 형태로 만듦
                answer_dict = {
                    "text": answer_text,
                    "answer_start": start_position_character
                }

                # 만약 is_impossible가 False라면 정답을 answers 리스트에 추가
                if not is_impossible:
                    answers.append(answer_dict)

                example = SquadExample(
                    qas_id=qas_id,
                    question_text=question_text,
                    context_text=context_text,
                    answer_text=answer_text,
                    start_position_character=start_position_character,
                    title=title,
                    is_impossible=is_impossible,
                    answers=answers
                )

                examples.append(example)
    print("input_data1 처리 완료")

    print("input_data2 처리 시작")
    for index, row in input_data2.iterrows():
      title = row['title']
      context_text = row["context"]
      qas_id = row["question_id"]
      question_text = row["question"]
      answer_text = row["answer"]
      start_position_character = int(row["answer_start"])
      end_position_character = int(row["answer_end"])
      answers = []

      is_impossible = row.get("is_impossible", False)

      # 정답 텍스트와 시작 위치를 가져와서 딕셔너리 형태로 만듦
      answer_dict = {
          "text": answer_text,
          "answer_start": start_position_character
      }

      # 만약 is_impossible가 False라면 정답을 answers 리스트에 추가
      if not is_impossible:
          answers.append(answer_dict)

      example = SquadExample(
          qas_id=qas_id,
          question_text=question_text,
          context_text=context_text,
          answer_text=answer_text,
          start_position_character=start_position_character,
          title=title,
          is_impossible=is_impossible,
          answers=answers
      )

      examples.append(example)

    print("input_data2 처리 완료")


    train_examples, test_examples = train_test_split(examples, test_size=0.2, random_state=42)
    return train_examples, test_examples

In [ ]:
def get_train_examples(filename1, filename2):

  if filename1 :
    path = Path(filename1)

    with open(path, "rb") as f:
      input_data1 = json.load(f)["data"]

  if filename2 :
    input_data2 = pd.read_csv(filename2)

  return create_examples(input_data1,input_data2, "train")

In [ ]:
filename1 = "/content/drive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/AI_hub_fin_data.json"
filename2 = "/content/drive/MyDrive/절미네 가족들/1.jiwon/1.data/0.row_data/QA_set_final_ver3.csv"

train_examples, test_examples = get_train_examples(filename1,filename2)

input_data1 처리 시작
input_data1 처리 완료
input_data2 처리 시작
input_data2 처리 완료


In [ ]:
title

1084

In [ ]:
len(train_examples)

15876